# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# Описание данных:

Данные геологоразведки трёх регионов находятся в файлах:

/datasets/geo_data_0.csv. 

/datasets/geo_data_1.csv.

/datasets/geo_data_2.csv. 

id — уникальный идентификатор скважины;

f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);

product — объём запасов в скважине (тыс. баррелей).

# Условия задачи:

Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).

При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.

Бюджет на разработку скважин в регионе — 10 млрд рублей.

При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

# 1. Загрузка и подготовка данных

In [1]:
#Подключим библиотеки
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

#Откроем 3 датасета
geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')
geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')

##Отобразим первые 5 строк каждого датасета
display('Датасет_0:', geo_data_0.head(5))
display('Датасет_1:', geo_data_1.head(5))
display('Датасет_2:', geo_data_2.head(5))

#Просмотрим общую информации о каждом датасете
print('Общая информация о Датасете_0:\n')
geo_data_0.info()

print('\nОбщая информация о Датасете_1:\n')
geo_data_1.info()

print('\nОбщая информация о Датасете_2:\n')
geo_data_2.info()


'Датасет_0:'

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


'Датасет_1:'

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


'Датасет_2:'

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Общая информация о Датасете_0:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

Общая информация о Датасете_1:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

Общая информация о Датасете_2:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 n

# 1.1 Вывод:

Загружено 3 датасета с данными. В каждом датасете 10000 строк и 5 столбцов. Все строки без нулевых значений. Столбец ID - это уникальны номер мтроки и для дальнейшей работы не нужен. Столбцы f0, f1, f2 - это признаки, они представлены как отрицательными значениями, так и положительными. Столцец product - это целевой признак и отображает объём запасов в скважине (тыс. баррелей).

# 2. Обучение и проверка модели

# 2.1 Обучение и проверка модели линейная регрессия для Датасета 0. 

In [2]:
#Определим признаки и целевой признак для модели линейная регрессия
target_0 = geo_data_0['product']
features_0 = geo_data_0.drop(['id', 'product'], axis=1)

#Выделим 25% тестовых данных из всего объема данных
features_train_0, features_valid_0, target_train_0, target_valid_0  = train_test_split(features_0, target_0, test_size=0.25, random_state=12345)

#Проверим модель линейной регрессии
model = LinearRegression()#создадим модель и установим гиперпараметры
model.fit(features_train_0, target_train_0) #обучим модель на тренировочной выборке
predicted_valid_0 = model.predict(features_valid_0) #предскажем на валидационной выборке
predicted_valid_0 = pd.Series(predicted_valid_0)

#вычислим RMSE
RMSE_0 = mean_squared_error(target_valid_0, predicted_valid_0)**0.5
print('RMSE Датасета_0:', RMSE_0)

#Вычислим средний запас предсказанного сырья
product_mean_0 = predicted_valid_0.mean()
print('Cредний запаc сырья Датасета_0:', product_mean_0, 'тыс. баррелей')

RMSE Датасета_0: 37.5794217150813
Cредний запаc сырья Датасета_0: 92.59256778438038 тыс. баррелей


# 2.2 Обучение и проверка модели линейная регрессия для Датасета 1. 

In [3]:
#Определим признаки и целевой признак для модели линейная регрессия
target_1 = geo_data_1['product']
features_1 = geo_data_1.drop(['id', 'product'], axis=1)

#Выделим 25% тестовых данных из всего объема данных
features_train_1, features_valid_1, target_train_1, target_valid_1  = train_test_split(features_1, target_1, test_size=0.25, random_state=12345)

#Проверим модель линейной регрессии
model = LinearRegression()#создадим модель и установим гиперпараметры
model.fit(features_train_1, target_train_1) #обучим модель на тренировочной выборке
predicted_valid_1 = model.predict(features_valid_1) #предскажем на валидационной выборке
predicted_valid_1 = pd.Series(predicted_valid_1)

#вычислим RMSE
RMSE_1 = mean_squared_error(target_valid_1, predicted_valid_1)**0.5
print('RMSE Датасета_1:', RMSE_1)

#Вычислим средний запас предсказанного сырья
product_mean_1 = predicted_valid_1.mean()
print('Cредний запаc сырья Датасета_1:', product_mean_1, 'тыс. баррелей')

RMSE Датасета_1: 0.893099286775616
Cредний запаc сырья Датасета_1: 68.728546895446 тыс. баррелей


# 2.3 Обучение и проверка модели линейная регрессия для Датасета 2. 

In [4]:
#Определим признаки и целевой признак для модели линейная регрессия
target_2 = geo_data_2['product']
features_2 = geo_data_2.drop(['id', 'product'], axis=1)

#Выделим 25% тестовых данных из всего объема данных
features_train_2, features_valid_2, target_train_2, target_valid_2  = train_test_split(features_2, target_2, test_size=0.25, random_state=12345)

#Проверим модель линейной регрессии
model = LinearRegression()#создадим модель и установим гиперпараметры
model.fit(features_train_2, target_train_2) #обучим модель на тренировочной выборке
predicted_valid_2 = model.predict(features_valid_2) #предскажем на валидационной выборке
predicted_valid_2 = pd.Series(predicted_valid_2)

#вычислим RMSE
RMSE_2 = mean_squared_error(target_valid_2, predicted_valid_2)**0.5 
print('RMSE Датасета_2:', RMSE_2)

#Вычислим средний запас предсказанного сырья
product_mean_2 = predicted_valid_2.mean()
print('Cредний запаc сырья Датасета_2:', product_mean_2, 'тыс. баррелей')

RMSE Датасета_2: 40.02970873393434
Cредний запаc сырья Датасета_2: 94.96504596800489 тыс. баррелей


# 2.4 Вывод:

Запасы сырья и их RMSE распределились мледующим образом:
- Датасет_0: RMSE: 37.5794217150813, средний запаc сырья: 92.59256778438038 тыс. баррелей
- Датасет_1: RMSE: 0.893099286775616, средний запаc сырья: 68.728546895446 тыс. баррелей
- Датасет_2: RMSE: 40.02970873393434, средний запаc сырья: 94.96504596800489 тыс. баррелей

Из полученных данных видно, что самый большой запас нефти находится в Датасете 2, потом Датасете 0 и самый маленький запас нефти в Датасете 1. Относительно RMSE самая низкая квадратичная ошибка в Датасете 1 и составляет 0.893, тогда как в Датасете 0 и Датасете 2 она гораздо выше и сотавляет 37.579 и 40.029. 

# 3. Подготовка к расчёту прибыли

# 3.1 Подготовим  ключевые значения для расчётов 

In [5]:
#Все ключевые значения для расчётов сохраним в отдельных переменных.
dots = 500 #При разведке региона исследуют 500 точек
best_dots = 200 #с помощью машинного обучения выбирают 200 лучших для разработки.
budget = 10000000000 #Бюджет на разработку скважин в регионе
profit = 450000 #Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
probability_loss = 2.5 #вероятность убытков

# 3.2 Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины

In [6]:
#Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины
#Исследуем 500 точек на них выделяется бюджет 10 млрд. и выбирается 200 лучших
#Поэтому вычислим сколько тратиться бюджета на 1 успешную скважину
budget_1_dot = budget/best_dots 

#Вычислим сколько нужно тысяч баррелей нефти, чтобы разработка была безубыточной для 1 точки:
count_barrel_1_dot = budget_1_dot / profit

print('Количество нефти для безубыточной разработки 1 точки:', count_barrel_1_dot, 'тыс. баррелей')

Количество нефти для безубыточной разработки 1 точки: 111.11111111111111 тыс. баррелей


# 3.3 Вывод:

Для безубыточной разработки 1 точки потребуется 111.1 тыс. баррелей нефти. Это значение больше, чем средний запас нефти в каждом из датасетов. На 13-15 % больше, чем средний запас в Датасете 0 и Датасете 2 и на 38% больше, чем в Датасете 1. 

# 4. Расчет прибыли

In [7]:
#Напишем функцию revenue(), которая подсчитывает и возвращает выручку
def revenue(target, predicted_valid, best_dots):
    probs_sorted = predicted_valid.sort_values(ascending=False)
    selected = target[probs_sorted.index][:best_dots]
    return profit * selected.sum() - budget

#Рассчитаем прибыль для 3-х Датасетов
profit_0 = revenue(target_0, predicted_valid_0, best_dots)
profit_1 = revenue(target_1, predicted_valid_1, best_dots)
profit_2 = revenue(target_2, predicted_valid_2, best_dots)
print('Прибыль для предсказаний Датасета_0:','{:.3f}'.format(profit_0/1000000000), 'млрд. руб.')
print('Прибыль для предсказаний Датасета_1:','{:.3f}'.format(profit_1/1000000000), 'млрд. руб.')
print('Прибыль для предсказаний Датасета_2:','{:.3f}'.format(profit_2/1000000000), 'млрд. руб.')


Прибыль для предсказаний Датасета_0: -1.922 млрд. руб.
Прибыль для предсказаний Датасета_1: -4.016 млрд. руб.
Прибыль для предсказаний Датасета_2: -1.259 млрд. руб.


# 4.1 Вывод:

Самый большой объем нефти содержится в предсказаных значениях Датасета 2, на втором месте Датасет 0 и на третьем месте Датасет 1. В таком же порядке расположена и прибыль, так как она прямо пропорциональна объемам сырья. Вся прибыль оказалась отрцательная. 

1. Прибыль для предсказаний Датасета_2: -1.259 млрд. руб.
2. Прибыль для предсказаний Датасета_0: -1.922 млрд. руб.
3. Прибыль для предсказаний Датасета_1: -4.016 млрд. руб.

# 5. Расчёт прибыли и рисков 

# 5.1 Расчёт прибыли и рисков для Датесета 0

In [8]:
#Напишем функцию bootstrap
state = np.random.RandomState(12345)  
values = []
for i in range(1000):
    target_subsample = target_0.sample(n=500, replace=True, random_state=state)
    probs_subsample = predicted_valid_0[target_subsample.index]
    values.append(revenue(target_subsample, probs_subsample, best_dots))

#Рассчитаем риск убытка
loss = 0
for value in predicted_valid_0:
    if value < 0:
        loss +=1
loss_predict = (loss / len(values)) * 100
        
#Определим границы доверительного интервала:
values = pd.Series(values)
lower = values.quantile(0.025)
upper = values.quantile(0.975)

#Определим среднюю выручку
mean = values.mean()

#Отобразим данные
print('Средняя выручка:','{:.3f}'.format(mean/1000000000), 'млрд. руб.')
print('Доверительный интервал Датасета_0:','{:.3f}'.format(lower/1000000000),'млрд. руб.', '-', '{:.3f}'.format(upper/1000000000), 'млрд. руб.')
print('Риск убытка Датасета_0:', loss_predict, '%' )

/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:1143: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


Средняя выручка: -1.647 млрд. руб.
Доверительный интервал Датасета_0: -2.216 млрд. руб. - -1.097 млрд. руб.
Риск убытка Датасета_0: 0.1 %


# 5.2 Расчёт прибыли и рисков для Датесета 1

In [9]:
#Напишем функцию bootstrap
state = np.random.RandomState(12345)  
values = []
for i in range(1000):
    target_subsample = target_1.sample(n=500, replace=True, random_state=state)
    probs_subsample = predicted_valid_1[target_subsample.index] 
    values.append(revenue(target_subsample, probs_subsample, best_dots))

#Рассчитаем риск убытка
loss = 0
for value in predicted_valid_1:
    if value < 0:
        loss +=1
loss_predict = (loss / len(values)) * 100
        
#Определим границы доверительного интервала:
values = pd.Series(values)
lower = values.quantile(0.025)
upper = values.quantile(0.975)

#Определим среднюю выручку
mean = values.mean()

#Отобразим данные
print('Средняя выручка:','{:.3f}'.format(mean/1000000000), 'млрд. руб.')
print('Доверительный интервал Датасета_1:','{:.3f}'.format(lower/1000000000),'млрд. руб.', '-', '{:.3f}'.format(upper/1000000000), 'млрд. руб.')
print('Риск убытка Датасета_1:', loss_predict, '%' )

Средняя выручка: -3.827 млрд. руб.
Доверительный интервал Датасета_1: -4.393 млрд. руб. - -3.283 млрд. руб.
Риск убытка Датасета_1: 53.900000000000006 %


# 5.3 Расчёт прибыли и рисков для Датесета 2

In [10]:
#Напишем функцию bootstrap
state = np.random.RandomState(12345)  
values = []
for i in range(1000):
    target_subsample = target_2.sample(n=500, replace=True, random_state=state)
    probs_subsample = predicted_valid_2[target_subsample.index] 
    values.append(revenue(target_subsample, probs_subsample, best_dots))

#Рассчитаем риск убытка
loss = 0
for value in predicted_valid_2:
    if value < 0:
        loss +=1
loss_predict = (loss / len(values)) * 100
        
#Определим границы доверительного интервала:
values = pd.Series(values)
lower = values.quantile(0.025)
upper = values.quantile(0.975)

#Определим среднюю выручку
mean = values.mean()

#Отобразим данные
print('Средняя выручка:','{:.3f}'.format(mean/1000000000), 'млрд. руб.')
print('Доверительный интервал Датасета_2:','{:.3f}'.format(lower/1000000000),'млрд. руб.', '-', '{:.3f}'.format(upper/1000000000), 'млрд. руб.')
print('Риск убытка Датасета_2:', loss_predict, '%' )

Средняя выручка: -1.439 млрд. руб.
Доверительный интервал Датасета_2: -2.006 млрд. руб. - -0.874 млрд. руб.
Риск убытка Датасета_2: 0.0 %


# 5.4 Вывод:

Датасет 0:
- Средняя выручка: -1.647 млрд. руб.
- Доверительный интервал Датасета_0: -2.216 млрд. руб. - -1.097 млрд. руб.
- Риск убытка Датасета_0: 0.1 %


Датасет 1:
- Средняя выручка: -3.827 млрд. руб.
- Доверительный интервал Датасета_1: -4.393 млрд. руб. - -3.283 млрд. руб.
- Риск убытка Датасета_1: 53.900000000000006 %

Датасет 2:
- Средняя выручка: -1.439 млрд. руб.
- Доверительный интервал Датасета_2: -2.006 млрд. руб. - -0.874 млрд. руб.
- Риск убытка Датасета_2: 0.0 %


Согласно полученным данным, видим, что выручка во всех 3-х датасетах отрицательная, в связи с чем невозможно выбрать ни один из регионов для дальнейшей разработки. 

# Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован